In [20]:
!pip install torchmetrics

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_us.UTF-8)


In [21]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from sklearn import metrics
from tqdm.notebook import tqdm

import mlflow

import csv
import json
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset
from torchmetrics import AveragePrecision

from tqdm.notebook import tqdm, trange
import tempfile

In [22]:
torch.__version__

'1.10.1+cu111'

In [23]:
print(f"¿CUDA disponible? {torch.cuda.is_available()}")

¿CUDA disponible? True


## Exploración de datos
Cargamos los datos de entrenamiento para ver que forma tienen

In [24]:
train_dataset = pd.read_json('./data/meli-challenge-2019/spanish.train.jsonl.gz',lines=True)
train_dataset.head(20)

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Casita Muñecas Barbies Pintadas,DOLLHOUSES,train,"[casita, muñecas, barbies, pintadas]","[50001, 2, 50000, 3]",0,632,4895280
1,spanish,unreliable,Neceser Cromado Holográfico,TOILETRY_BAGS,train,"[neceser, cromado, holográfico]","[6, 4, 5]",1,632,4895280
2,spanish,unreliable,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS,train,"[funda, asiento, medida, chevrolet]","[9, 7, 10, 8]",2,632,4895280
3,spanish,unreliable,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS,train,"[embrague, ford, focus, one]","[11, 13, 12, 14]",3,632,4895280
4,spanish,unreliable,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES,train,"[bateria, panasonic, dmwbcf, lumix, dmc, fxn, ...","[15, 19, 17, 18, 16, 1, 1, 1]",4,632,4895280
5,spanish,unreliable,Gurgel Br 800,CLASSIC_CARS,train,[gurgel],[1],5,632,4895280
6,spanish,unreliable,Harman Kardon Hk 3700 2ch Network Receiver _h,AV_RECEIVERS,train,"[harman, kardon, network, receiver]","[20, 21, 22, 23]",6,632,4895280
7,spanish,unreliable,Pack Netbook´s,POWER_GRINDERS,train,"[pack, netbook´s]","[24, 1]",7,632,4895280
8,spanish,unreliable,Olla Essen Duo,KITCHEN_POTS,train,"[olla, essen, duo]","[27, 26, 25]",8,632,4895280
9,spanish,unreliable,Teclado Mini Bluetooth,PC_KEYBOARDS,train,"[teclado, mini, bluetooth]","[30, 29, 28]",9,632,4895280


In [25]:
train_dataset.language.value_counts()

spanish    4895280
Name: language, dtype: int64

In [26]:
train_dataset.label_quality.value_counts()

unreliable    4508043
reliable       387237
Name: label_quality, dtype: int64

## Dataset

Tomado de https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb


In [27]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None, key = 'title'):
        """
        path: Ubicación a los datos (comprimidos con gzip)
        key: Columna que vamos a usar para entrenar
        """
        self.dataset_path = path
        self.transform = transform
        self.key = key

    def __iter__(self):
        """
        Habilita un iterador sobre los datos
        """
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data[self.key],
                    "target": data['target']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item


In [28]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]
        

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
#            "data": torch.LongTensor(data),
            "data": torch.FloatTensor(data), 
            "target": torch.FloatTensor(target)
        }

In [29]:
# Definimos una collate_fn que nos retorne todos los arreglos de la misma longitud
data_len = train_dataset.data.apply(lambda v: len(v))
max_len = data_len.max()

pad_to_len = PadSequences(max_length = max_len)

### Juntando todo
Tenemos
* Una clase con la responsabilidad de entregar datos, potencialmente preprocesandolos si hace falta
* Una función que transforma los datos leidos para que todos los valores tengan la misma longitud

A partir de esto, creamos dos instancia del `DataLoader`: Uno para cargar los datos de entrenamiento y otro para cargar los datos de _test_

In [30]:
dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.train.jsonl.gz', key = 'data')
train_dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn = pad_to_len)

In [31]:
dataset = MeLiChallengeDataset('./data/meli-challenge-2019/spanish.test.jsonl.gz', key = 'data')
test_dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn = pad_to_len)

## MLP

Tomamos el MLP de las clases y agregamos los parametros que nos interesan

* Capa de entrada: Tantas neuronas como tokens
* Capa de salida: Tantas neuronas como categorías (632)

In [32]:
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden = []):
        """
        input_size: Número de neuronas de entrada
        output_size: Número de neuronas de salida
        hidden: Lista con los numeros de capas ocultas
        """
        super().__init__()
        
        neurons = [input_size]  + hidden 
        parts = []
        for idx, each in enumerate(neurons[:-1]):
            parts.append(nn.Linear(each, neurons[idx + 1]))
            parts.append(nn.ReLU())
            
        parts = parts + [nn.Linear(neurons[-1], 1), nn.Sigmoid()]
        
        self.model = nn.Sequential(*parts)
    
    def forward(self, x: torch.Tensor):
        return self.model(x)

In [33]:
model = MLP(20, 632, hidden= [2])
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [34]:
t = torch.Tensor(1,20)
model(t)

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [35]:
model.train()

MLP(
  (model): Sequential(
    (0): Linear(in_features=20, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [36]:

for idx, batch in enumerate(tqdm(train_dataloader)):
    print(model(batch["data"]))

0it [00:00, ?it/s]

tensor([[1.0000],
        [0.4196],
        [0.5226],
        [0.6446]], grad_fn=<SigmoidBackward0>)
tensor([[0.5455],
        [0.4005],
        [0.7682],
        [0.4177]], grad_fn=<SigmoidBackward0>)
tensor([[0.5807],
        [0.6010],
        [0.8814],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.7178],
        [0.7296],
        [0.5565],
        [0.9810]], grad_fn=<SigmoidBackward0>)
tensor([[0.5909],
        [0.9403],
        [0.6431],
        [0.8670]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.9983],
        [0.9987],
        [0.9990]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.9994],
        [0.7819],
        [0.7808]], grad_fn=<SigmoidBackward0>)
tensor([[0.7318],
        [0.9996],
        [0.9652],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [0.9988],
        [0.9751]], grad_fn=<SigmoidBackward0>)
tensor([[0.9970],
        [1.0000],
        [0.9792],
        [0.8961]], grad_fn=<SigmoidBa

tensor([[1.0000],
        [0.3853],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
    

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.9158],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor

tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.9728],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.9802],
        [0.3853],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor

...

tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
  

tensor([[0.3853],
        [1.0000],
        [0.9998],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.9660],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.5796],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
    

tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor

tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [0.3853],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [0.3853],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [0.3853]], grad_fn=<SigmoidBackward0>)
tensor([[0.3853],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000],
        [1.0000],
        [0.3853],
        [1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)
tensor([[1.],
        

KeyboardInterrupt: 

In [24]:

for idx, batch in enumerate(tqdm(train_dataloader)):
    optimizer.zero_grad()
    output = model(batch["data"])
    loss_value = loss(output, batch["target"].view(-1, 1))
    loss_value.backward()
    optimizer.step()

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
#                           collate_fn=pad_sequences, drop_last=False)
# test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
#                          collate_fn=pad_sequences, drop_last=False)

In [41]:
pred = torch.tensor([[0.75, 0.05, 0.05, 0.05, 0.05],
                     [0.05, 0.75, 0.05, 0.05, 0.05],
                     [0.05, 0.05, 0.75, 0.05, 0.05],
                     [0.05, 0.05, 0.05, 0.75, 0.05]])
target = torch.tensor([0, 1, 3, 2])
AveragePrecision(num_classes=5, average='weighted')

average_precision(pred, target).item()

torch.tensor(pred.squeeze().detach().numpy())


tensor([[0.7500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.7500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.7500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.7500, 0.0500]])

In [42]:
mlflow.set_experiment("MLP Basico")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp") # TODO Log parameters
    mlflow.log_param("epochs", "1")
    mlflow.log_param("hidden_layer_1_neurons", "2")
    
    model = MLP(20, 632, hidden= [2]) # 20 -> 2 -> 632
    loss = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    
    for epoch in trange(1): # TODO Pruebas con varias epochs
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_dataloader)):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss(output, batch["target"].view(-1, 1))
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_dataloader):
            output = model(batch["data"])
            running_loss.append(
                loss(output, batch["target"].view(-1, 1)).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        # Reemplazamos la siguiente metrica por la de pythorch (La de scikit learn no acepta problemas multiclase)
        # mlflow.log_metric("test_avp", average_precision_score(targets, predictions), epoch)
        
        # https://torchmetrics.readthedocs.io/en/stable/classification/average_precision.html
        # Computes the average precision score, which summarises the precision recall curve into one number. 
        # Works for both binary and multiclass problems. 
        # In the case of multiclass, the values will be calculated based on a one- vs-the-rest approach.
        # 'weighted': Calculate the metric for each class separately, and average the metrics across classes, 
        # weighting each class by its support.
        
        # TODO Probar que funcione
        # avp = AveragePrecision(num_classes = 632, average = 'weighted')
        # mlflow.log_metric("test_avp", avp(torch.tensor(targets), torch.tensor(predictions)).item(), epoch)
        


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

IndexError: tuple index out of range